In [1]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
# import yaml
# import re

# import pyarrow as pa
# import pyarrow.parquet as pq

# import gbd_mapping as gm
from vivarium import InteractiveContext
import scipy

# from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
# from get_draws.api import get_draws

# import vivarium_helpers as vh
# import vivarium_helpers.id_helper as idh
# from vivarium_helpers.vph_output.operations import VPHOperator
# from vivarium_helpers.vph_output import cleaning

!date
!whoami
!pwd

Tue Nov  4 15:45:30 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/verification_and_validation


In [2]:
! pip list | grep vivarium

vivarium                  3.6.0
vivarium_build_utils      2.0.12
vivarium_cluster_tools    2.1.23
vivarium_csu_alzheimers   0.1.dev58+g263eb3d9b /mnt/share/code/ndbs/vivarium_csu_alzheimers
vivarium_dependencies     1.0.1
vivarium_public_health    4.3.12


In [3]:
! pip freeze | grep vivarium

vivarium==3.6.0
vivarium_build_utils==2.0.12
vivarium_cluster_tools==2.1.23
-e git+https://github.com/ihmeuw/vivarium_csu_alzheimers.git@263eb3d9b80b13735a7380bb892430eaba92603a#egg=vivarium_csu_alzheimers
vivarium_dependencies==1.0.1
vivarium_public_health==4.3.12


# Automatically find the path to the model spec file

Code from [Vivarium Research documentation of interactive
simulations](https://vivarium-research.readthedocs.io/en/latest/model_design/designing_vivarium_model/interactive_sim/index.html).

See also the [interactive simulation tutorial in the Vivarium docs](https://vivarium.readthedocs.io/en/latest/tutorials/running_a_simulation/interactive.html).

In [4]:
import vivarium_csu_alzheimers
# model_specifications/model_spec.yaml is the typical location within the engineering repo for the
# default spec to live. Ask the engineers if you can't find it.
path = Path(vivarium_csu_alzheimers.__file__.replace('__init__.py', 'model_specifications/model_spec.yaml'))
print(path)

!cat $path

/mnt/share/code/ndbs/vivarium_csu_alzheimers/src/vivarium_csu_alzheimers/model_specifications/model_spec.yaml
components:
    vivarium_csu_alzheimers:
      components:
          - ResultsStratifier()
          - NewSimulantsObserver()
          - AlzheimersIncidence()
          - AlzheimersPopulation('population.scaling_factor')
          - Alzheimers()
          - Testing()
          - BaselineTestingObserver()
          - BBBMTestingObserver()
          - Treatment()
          - TreatmentObserver()
          - TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')
            
    vivarium_public_health:
        population:
            - Mortality()
        results:
            - DisabilityObserver()
            - MortalityObserver()
            - DiseaseObserver('alzheimers_disease_and_other_dementias')

configuration:
    input_data:
        input_draw_number: 0
        artifact_path: '/mnt/team/simulation_

# Customize the model specification by deleting observers

In [5]:
from vivarium.framework.configuration import build_model_specification
custom_model_specification = build_model_specification(path)

# Delete mortality, morbidity and disease observers
print(custom_model_specification.components.vivarium_public_health.results)
del custom_model_specification.components.vivarium_public_health.results

# Filter out components with 'Observer' or 'Stratifier' in the name
observers = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if 'Observer' in c or 'Stratifier' in c
]
print(observers)
custom_model_specification.components.vivarium_csu_alzheimers.components = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if c not in observers
]

# Set scenario to intervention instead of baseline
custom_model_specification.configuration.intervention.scenario = 'bbbm_testing_and_treatment'

custom_model_specification.components.vivarium_csu_alzheimers.components


['DisabilityObserver()', 'MortalityObserver()', "DiseaseObserver('alzheimers_disease_and_other_dementias')"]
['ResultsStratifier()', 'NewSimulantsObserver()', 'BaselineTestingObserver()', 'BBBMTestingObserver()', 'TreatmentObserver()']


['AlzheimersIncidence()',
 "AlzheimersPopulation('population.scaling_factor')",
 'Alzheimers()',
 'Testing()',
 'Treatment()',
 "TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')"]

# Here's an alternative way to get a model spec and create a sim

In [7]:
location = "United States of America"
project_directory = Path("/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/")
model_spec_path = (
    project_directory /
    "results/model8.5/united_states_of_america/2025_11_04_08_09_23/model_specification.yaml"
)
artifact_path = (
    project_directory /
    f"artifacts/model8.3/{location.lower().replace(' ', '_')}.hdf"
)

sim1 = InteractiveContext(model_spec_path, setup=False)
sim1.configuration.update(
    {
        'input_data': {'artifact_path': artifact_path},
        'intervention': {'scenario': 'bbbm_testing_and_treatment'},
    }
)
sim1.setup()

2025-11-04 15:47:41.482 | INFO     | simulation_2-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf.
2025-11-04 15:47:41.483 | INFO     | simulation_2-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-11-04 15:47:41.484 | INFO     | simulation_2-artifact_manager:81 - Artifact additional filter terms are None.
2025-11-04 15:47:44.318 | INFO     | simulation_2-results_context:129 - The following stratifications are registered but not used by any observers: 
['current_year', 'semester']
2025-11-04 15:47:44.484 | WARNING  | simulation_2-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2025-11-04 15:47:44.485 | WARNING  | simulation_2-resource_manager:176 - Resource stream.treatment_initial_sta

# Create a sim using custom model specification

In [8]:
sim = InteractiveContext(custom_model_specification)

2025-11-04 15:47:53.646 | INFO     | simulation_3-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf.
2025-11-04 15:47:53.647 | INFO     | simulation_3-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-11-04 15:47:53.648 | INFO     | simulation_3-artifact_manager:81 - Artifact additional filter terms are None.
2025-11-04 15:47:56.135 | WARNING  | simulation_3-values_manager:55 - Unsourced pipelines: ['all_causes.disability_weight']
2025-11-04 15:47:56.140 | WARNING  | simulation_3-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2025-11-04 15:47:56.141 | WARNING  | simulation_3-resource_manager:176 - Resource stream.treatment_initial_states is not produced by any component but is needed to 

In [9]:
pop = sim.get_population()
pop

,tracked,alive,age,exit_time,entrance_time,location,sex,years_of_life_lost,cause_of_death,alzheimers_disease_and_other_dementias,...,full_effect_short_event_count,full_effect_short_event_time,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_time,waning_effect_short_event_count,no_effect_after_long_event_time,no_effect_after_long_event_count,no_effect_after_short_event_count,no_effect_after_short_event_time
0,True,alive,79.295947,NaT,2021-07-03,United States of America,Male,0.0,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
1,True,alive,88.728513,NaT,2021-07-03,United States of America,Male,0.0,not_dead,alzheimers_mild_cognitive_impairment_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
2,True,alive,93.193052,NaT,2021-07-03,United States of America,Female,0.0,not_dead,alzheimers_mild_cognitive_impairment_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
3,True,alive,83.653926,NaT,2021-07-03,United States of America,Female,0.0,not_dead,alzheimers_disease_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
4,True,alive,66.246647,NaT,2021-07-03,United States of America,Male,0.0,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,True,alive,82.436768,NaT,2021-07-03,United States of America,Female,0.0,not_dead,alzheimers_disease_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
19996,True,alive,75.910630,NaT,2021-07-03,United States of America,Male,0.0,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
19997,True,alive,76.259348,NaT,2021-07-03,United States of America,Female,0.0,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
19998,True,alive,78.185919,NaT,2021-07-03,United States of America,Female,0.0,not_dead,alzheimers_mild_cognitive_impairment_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT


# Print list of columns and list of pipeline values

In [10]:
pop.columns

Index(['tracked', 'alive', 'age', 'exit_time', 'entrance_time', 'location',
       'sex', 'years_of_life_lost', 'cause_of_death',
       'alzheimers_disease_and_other_dementias', 'bbbm_entrance_time',
       'alzheimers_blood_based_biomarker_state_event_time',
       'alzheimers_blood_based_biomarker_state_event_count',
       'alzheimers_mild_cognitive_impairment_state_event_count',
       'alzheimers_mild_cognitive_impairment_state_event_time',
       'alzheimers_disease_state_event_count',
       'alzheimers_disease_state_event_time', 'bbbm_test_ever_eligible',
       'bbbm_test_date', 'bbbm_test_result', 'testing_state',
       'testing_propensity', 'treatment_propensity',
       'no_effect_never_treated_event_time',
       'waiting_for_treatment_event_time', 'waiting_for_treatment_event_count',
       'treatment', 'no_effect_never_treated_event_count',
       'susceptible_to_treatment_event_time',
       'susceptible_to_treatment_event_count', 'positive_test_event_time',
       'p

In [11]:
sim.list_values()

['simulant_step_size',
 'cause_specific_mortality_rate',
 'alzheimers_blood_based_biomarker_state.dwell_time',
 'alzheimers_blood_based_biomarker_state.disability_weight',
 'all_causes.disability_weight',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate.paf',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate',
 'mortality_rate',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate',
 'alzheimers_mild_cognitive_impairment_state.dwell_time',
 'alzheimers_mild_cognitive_impairment_state.disability_weight',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate.paf',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate.paf',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_s

In [12]:
[v for v in sim.list_values() if 'rate' in v]

['cause_specific_mortality_rate',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate.paf',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate',
 'mortality_rate',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate.paf',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate.paf',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate',
 'alzheimers_disease_state.excess_mortality_rate.paf',
 'alzheimers_disease_state.excess_mortality_rate',
 'waiting_for_treatment.excess_mortality_rate.paf',
 'waiting_for_treatment.excess_mortality_rate',
 'full_effect_long.excess_mortality_rate.paf',
 'full_effect_long

In [13]:
pop['alzheimers_disease_and_other_dementias']

0            alzheimers_blood_based_biomarker_state
1        alzheimers_mild_cognitive_impairment_state
2        alzheimers_mild_cognitive_impairment_state
3                          alzheimers_disease_state
4            alzheimers_blood_based_biomarker_state
                            ...                    
19995                      alzheimers_disease_state
19996        alzheimers_blood_based_biomarker_state
19997        alzheimers_blood_based_biomarker_state
19998    alzheimers_mild_cognitive_impairment_state
19999    alzheimers_mild_cognitive_impairment_state
Name: alzheimers_disease_and_other_dementias, Length: 20000, dtype: object

In [14]:
s = sim.get_value('alzheimers_mild_cognitive_impairment_state.dwell_time')(pop.index)
s.loc[s>0]

Series([], Name: alzheimers_mild_cognitive_impairment_state.dwell_time, dtype: float64)

In [15]:
s = sim.get_value('alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf')(pop.index)
s.loc[s>0]

Series([], Name: 0, dtype: float64)

# Check dwell times for treatment states

Looks good.

In [16]:
[v for v in sim.list_values() if 'dwell_time' in v]

['alzheimers_blood_based_biomarker_state.dwell_time',
 'alzheimers_mild_cognitive_impairment_state.dwell_time',
 'alzheimers_disease_state.dwell_time',
 'waiting_for_treatment.dwell_time',
 'full_effect_long.dwell_time',
 'full_effect_short.dwell_time',
 'waning_effect_long.dwell_time',
 'waning_effect_short.dwell_time',
 'no_effect_after_long.dwell_time',
 'no_effect_after_short.dwell_time',
 'no_effect_never_treated.dwell_time']

In [18]:
# 1 time step = 182 days
sim.get_value('waiting_for_treatment.dwell_time')(pop.index).unique()

array([182.])

In [ ]:
# Divide by 2 * 182 to covert to years
sim.get_value('full_effect_long.dwell_time')(pop.index).unique() / (2 * 182)

array([5.])

In [20]:
sim.get_value('full_effect_short.dwell_time')(pop.index).unique()

array([182.])

In [21]:
# Divide by 2 * 182 to convert to years
sim.get_value('waning_effect_long.dwell_time')(pop.index).unique() / (2 * 182)

array([9.])

In [24]:
# Divide by 2 * 182 to convert to years
sim.get_value('waning_effect_short.dwell_time')(pop.index).unique() / (2 * 182)

array([2.5])

In [23]:
sim.get_value('no_effect_after_long.dwell_time')(pop.index).unique()

array([0.])

# Advance the population to a time when some people have treatment

In [24]:
sim.run_until(pd.Timestamp('2047-12-31'))

2025-11-04 15:56:47.383 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-01-01 00:00:00
2025-11-04 15:56:52.862 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-07-02 00:00:00
2025-11-04 15:56:58.581 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-12-31 00:00:00
2025-11-04 15:57:03.667 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2023-07-01 00:00:00
2025-11-04 15:57:09.412 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2023-12-30 00:00:00
2025-11-04 15:57:15.248 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2024-06-29 00:00:00
2025-11-04 15:57:21.936 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2024-12-28 00:00:00
2025-11-04 15:57:27.912 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2025-06-28 00:00:00
2025-11-04 15:57:33.295 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2025-12-27 00:00:00
2025-11-04 15:57:39.537 | INFO     | simulation_3 - vivarium.fra

In [25]:
pop = sim.get_population()
pop

,tracked,alive,age,exit_time,entrance_time,location,sex,years_of_life_lost,cause_of_death,alzheimers_disease_and_other_dementias,...,full_effect_short_event_count,full_effect_short_event_time,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_time,waning_effect_short_event_count,no_effect_after_long_event_time,no_effect_after_long_event_count,no_effect_after_short_event_count,no_effect_after_short_event_time
0,True,dead,87.268568,2030-06-22,2021-07-03,United States of America,Male,9.936408,other_causes,alzheimers_disease_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
1,True,dead,91.219958,2024-12-28,2021-07-03,United States of America,Male,8.727986,other_causes,alzheimers_mild_cognitive_impairment_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
2,True,dead,96.681074,2025-12-27,2021-07-03,United States of America,Female,8.302659,other_causes,alzheimers_mild_cognitive_impairment_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
3,True,dead,84.152214,2022-12-31,2021-07-03,United States of America,Female,11.466826,other_causes,alzheimers_disease_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
4,True,dead,71.727825,2027-12-25,2021-07-03,United States of America,Male,20.675717,other_causes,alzheimers_mild_cognitive_impairment_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124174,True,alive,92.211874,NaT,2047-11-30,NaN,Male,0.000000,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
124175,True,alive,94.894029,NaT,2047-11-30,NaN,Male,0.000000,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
124176,True,alive,93.454686,NaT,2047-11-30,NaN,Male,0.000000,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT
124177,True,alive,91.215896,NaT,2047-11-30,NaN,Male,0.000000,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,NaT,0,NaT,0,NaT,0,0,NaT


In [26]:
sim.get_value('treatment.exposure')(pop.index).unique()

array(['susceptible_to_treatment', 'no_effect_never_treated',
       'waiting_for_treatment', 'waning_effect_long',
       'no_effect_after_long', 'full_effect_long',
       'no_effect_after_short', 'waning_effect_short',
       'full_effect_short'], dtype=object)

In [27]:
# Why are some relative risks > 1.0?
# I think it only happens for dead simulants -- we'll check later
sim.get_value(
    'treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk')(pop.index).unique()

array([1.        , 1.20999293, 0.52751591, 1.05249823, 1.1574947 ,
       1.07874735, 0.79000707, 1.13124558, 0.55376502, 1.10499647,
       0.97375088, 0.92125265, 1.18374381, 0.6850106 , 0.63251237,
       0.73750884, 0.94750177, 0.58001414, 0.89500353, 0.81625619,
       1.02624912, 0.86875442, 3.36242047, 0.8425053 , 0.76375795,
       3.83490456, 0.60626326, 0.71125972, 3.07893001, 3.17342683,
       0.65876149, 2.79543956, 2.13396182, 2.60644592, 0.81100636,
       2.5119491 , 2.70094274, 1.94496819, 1.66147773, 2.32295546,
       1.18899364, 2.03946501, 1.75597455, 1.56698091, 0.90550318,
       1.28349046, 0.71650954, 0.62201273])

In [28]:
sim.current_time

Timestamp('2048-05-30 00:00:00')

# Define a function to get columns and pipeline values we want from the population

In [29]:
# List columns we want to keep
columns = [
    'age', 'alive', 'bbbm_test_result', 'treatment',
    'alzheimers_disease_and_other_dementias',
    'bbbm_entrance_time',
]
# Rename some columns with shorter names if desired
columns_to_short_names = {c: c for c in columns} | {
    'alzheimers_disease_and_other_dementias': 'ad_state',
}
# Simultaneously list pipeline values we want and map them to shorter
# names
values_to_short_names = {
    'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate':
        'mci_inc_rate',
    'treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk':
        'relative_risk',
}

def get_pop_columns(
        sim,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Get the specified subset of population columns concatenated with
    the specified pipeline values, with columns for pipeline values
    renamed according to the names in the provided "short names"
    dictionary.
    """
    pop = sim.get_population()
    pipeline_values = [
        sim.get_value(v)(pop.index).rename(short_name)
        for v, short_name in values_to_short_names.items()]
    pop_columns = (
        pop[columns_to_short_names.keys()]
        .rename(columns=columns_to_short_names)
        .join(pipeline_values)
        # Add current time to the dataframe
        .assign(current_time=sim.current_time)
    )
    return pop_columns

def take_steps(
        sim,
        n,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Take n simulation steps, recording the specified population
    columns and values at each step, and return a list of the population
    tables at each step. The resulting list will have n+1 population
    tables.
    """
    population_trace = [
        get_pop_columns(sim, columns_to_short_names, values_to_short_names)]
    for _ in range(n):
        sim.step()
        population_trace.append(get_pop_columns(
            sim, columns_to_short_names, values_to_short_names))
    return population_trace

pop1 = get_pop_columns(sim)
pop1

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.123728,1.0,2048-05-30
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.122090,1.0,2048-05-30
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.126285,1.0,2048-05-30
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.126686,1.0,2048-05-30
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-01-03 12:31:22.341663442,0.127057,1.0,2048-05-30
...,...,...,...,...,...,...,...,...,...
124174,92.211874,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124175,94.894029,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124176,93.454686,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124177,91.215896,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30


# See what the relative risks look like when they're not 1.0

In [30]:
pop1.query("relative_risk != 1.0")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
1708,87.869944,dead,positive,waning_effect_long,alzheimers_mild_cognitive_impairment_state,2018-12-10 02:01:40.522973563,0.150562,1.209993,2048-05-30
12016,88.191754,dead,positive,waning_effect_long,alzheimers_mild_cognitive_impairment_state,2021-07-22 21:49:29.982230094,0.147513,1.209993,2048-05-30
13502,90.127423,dead,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2018-06-23 19:55:25.556567324,0.151080,1.209993,2048-05-30
16135,80.645822,dead,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2021-05-05 23:06:05.019770534,0.064423,0.527516,2048-05-30
19521,84.616974,dead,positive,waning_effect_long,alzheimers_disease_state,2017-08-23 21:46:22.742481443,0.151994,1.209993,2048-05-30
...,...,...,...,...,...,...,...,...,...
121549,80.805957,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01 00:00:00.000000000,0.031159,0.527516,2048-05-30
121676,79.992750,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01 00:00:00.000000000,0.031159,0.527516,2048-05-30
121678,79.590437,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01 00:00:00.000000000,0.031159,0.527516,2048-05-30
121694,80.236739,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01 00:00:00.000000000,0.031159,0.527516,2048-05-30


# Test the take_steps function

In [31]:
pop_trace = take_steps(sim, 4)
len(pop_trace)

2025-11-04 16:06:47.479 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2048-05-30 00:00:00
2025-11-04 16:07:03.328 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2048-11-28 00:00:00
2025-11-04 16:07:18.182 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2049-05-29 00:00:00
2025-11-04 16:07:34.054 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2049-11-27 00:00:00


5

In [32]:
pop_trace[0]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.123728,1.0,2048-05-30
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.122090,1.0,2048-05-30
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.126285,1.0,2048-05-30
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.126686,1.0,2048-05-30
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-01-03 12:31:22.341663442,0.127057,1.0,2048-05-30
...,...,...,...,...,...,...,...,...,...
124174,92.211874,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124175,94.894029,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124176,93.454686,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124177,91.215896,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30


In [33]:
pop_trace[1]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.124197,1.0,2048-11-28
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.122582,1.0,2048-11-28
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.126722,1.0,2048-11-28
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.127118,1.0,2048-11-28
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-01-03 12:31:22.341663442,0.127485,1.0,2048-11-28
...,...,...,...,...,...,...,...,...,...
126274,93.013379,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28
126275,91.094237,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28
126276,92.981823,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28
126277,91.072722,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28


In [34]:
pop_trace[2]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.124660,1.0,2049-05-29
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.123066,1.0,2049-05-29
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.127153,1.0,2049-05-29
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.127545,1.0,2049-05-29
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-01-03 12:31:22.341663442,0.127907,1.0,2049-05-29
...,...,...,...,...,...,...,...,...,...
128421,87.041272,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-11-28 00:00:00.000000000,0.050712,1.0,2049-05-29
128422,87.952956,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-11-28 00:00:00.000000000,0.050712,1.0,2049-05-29
128423,88.152936,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-11-28 00:00:00.000000000,0.050712,1.0,2049-05-29
128424,95.115212,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-11-28 00:00:00.000000000,0.050712,1.0,2049-05-29


# Define Weibull distribution and hazard function

We'll use this to check the MCI incidence rate.

In [35]:
λ=6.76
k=1.22

weibull_dist = scipy.stats.weibull_min(k, scale=λ)
weibull_dist.cdf([5, 10])

array([0.49950771, 0.8005858 ])

In [36]:
def hazard(t, dist):
    # hazard = (probability density) / (survival function)
    return dist.pdf(t) / dist.sf(t)

def hazard_with_logs(t, dist):
    # Take logs first to avoid underflow
    return np.exp(dist.logpdf(t) - dist.logsf(t))

mci_hazard_func = lambda t: hazard_with_logs(t, weibull_dist)

In [37]:
DAYS_PER_YEAR = 365.25

def years_in_bbbm(current_time, bbbm_entrance_time):
    return (current_time - bbbm_entrance_time).dt.days / DAYS_PER_YEAR

def mci_incidence_hazard(pop):
    hazard = mci_hazard_func(
        years_in_bbbm(pop['current_time'], pop['bbbm_entrance_time']))
    return pd.Series(hazard, index=pop.index, name='mci_hazard')

mci_incidence_hazard(pop_trace[1])

0         0.249035
1         0.245798
2         0.254101
3         0.254895
4         0.255629
            ...   
126274    0.101688
126275    0.101688
126276    0.101688
126277    0.101688
126278    0.101688
Name: mci_hazard, Length: 126279, dtype: float64

# Check that calculated Weibull hazard is increasing over time

Yes, looks good.

In [38]:
pd.concat([mci_incidence_hazard(pop) * pop['relative_risk'] for pop in pop_trace], axis=1)

,0,1,2,3,4
0,0.248094,0.249035,0.249963,0.250879,0.251784
1,0.244813,0.245798,0.246770,0.247729,0.248675
2,0.253226,0.254101,0.254966,0.255821,0.256665
3,0.254030,0.254895,0.255751,0.256596,0.257432
4,0.254772,0.255629,0.256476,0.257313,0.258140
...,...,...,...,...,...
132618,NaN,NaN,NaN,NaN,0.101688
132619,NaN,NaN,NaN,NaN,0.101688
132620,NaN,NaN,NaN,NaN,0.101688
132621,NaN,NaN,NaN,NaN,0.101688


# Add the calculated MCI incidence hazard to each population in the trace

In [ ]:
STEP_SIZE_IN_YEARS = 182 / 365 # Not sure what value is used in the sim
pops = [
    pop.assign(
        mci_hazard=mci_incidence_hazard(pop),
        mci_prob=lambda df: df['mci_hazard'] * STEP_SIZE_IN_YEARS,
    )
    for pop in pop_trace]
pops[0]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.123728,1.0,2048-05-30,0.248094,0.123707
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.122090,1.0,2048-05-30,0.244813,0.122071
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.126285,1.0,2048-05-30,0.253226,0.126266
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.126686,1.0,2048-05-30,0.254030,0.126667
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-01-03 12:31:22.341663442,0.127057,1.0,2048-05-30,0.254772,0.127037
...,...,...,...,...,...,...,...,...,...,...,...
124174,92.211874,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124175,94.894029,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124176,93.454686,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124177,91.215896,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705


In [ ]:
# These ratios should all be equal to or close to 1
pops[0]['mci_prob'] / pops[0]['mci_inc_rate']

0         0.999833
1         0.999843
2         0.999848
3         0.999846
4         0.999840
            ...   
124174    0.999849
124175    0.999849
124176    0.999849
124177    0.999849
124178    0.999849
Length: 124179, dtype: float64

In [ ]:
# Why is a year equal to approximately 364.945023 days??
pops[0]['mci_hazard'] * 182 / 364.945023 / pops[0]['mci_inc_rate']

0         0.999984
1         0.999994
2         0.999999
3         0.999997
4         0.999991
            ...   
124174    1.000000
124175    1.000000
124176    1.000000
124177    1.000000
124178    1.000000
Length: 124179, dtype: float64

# It looks like the calculated Weibull hazard now matches the incidence rate in the sim

The `mci_inc_rate` column should be the _probability_ of transitioning,
i.e., the hazard times the step size, which is about 0.5. When I compute
the probability based on the calculated Weibull hazard, they are close.

In [55]:
pops[0].query("alive=='alive'")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
8,77.629473,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-08-16 15:35:15.636560159,0.126519,1.0,2048-05-30,0.253694,0.126499
53,82.836929,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2015-12-28 15:08:39.188644365,0.127071,1.0,2048-05-30,0.254801,0.127051
83,80.191280,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-04-07 22:24:17.131821007,0.124129,1.0,2048-05-30,0.248901,0.124110
84,75.742074,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2018-08-06 00:37:46.649681279,0.124751,1.0,2048-05-30,0.250145,0.124730
115,83.786217,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2017-02-27 09:52:59.458093924,0.126049,1.0,2048-05-30,0.252750,0.126029
...,...,...,...,...,...,...,...,...,...,...,...
124174,92.211874,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124175,94.894029,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124176,93.454686,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124177,91.215896,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705


In [56]:
pops[1].query("alive=='alive'")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
8,78.127762,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-08-16 15:35:15.636560159,0.126953,1.0,2048-11-28,0.254564,0.126933
83,80.689569,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-04-07 22:24:17.131821007,0.124592,1.0,2048-11-28,0.249831,0.124573
84,76.240363,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2018-08-06 00:37:46.649681279,0.125207,1.0,2048-11-28,0.251059,0.125186
115,84.284506,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2017-02-27 09:52:59.458093924,0.126489,1.0,2048-11-28,0.253631,0.126468
144,80.116992,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2015-09-23 21:31:31.817190720,0.127721,1.0,2048-11-28,0.256105,0.127702
...,...,...,...,...,...,...,...,...,...,...,...
126274,93.013379,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28,0.101688,0.050705
126275,91.094237,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28,0.101688,0.050705
126276,92.981823,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28,0.101688,0.050705
126277,91.072722,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30 00:00:00.000000000,0.050712,1.0,2048-11-28,0.101688,0.050705


# Check where the relative risk is not 1.0

The RR should always be < 1.0. For simulants in one of the two "full
effect" states, the RR should always be equal to the uniform draw from
[0.4, 0.6] -- we could check the specific draw in the artifact if we
wanted. For simulants in one of the two "waning effect" states, the RR
should be somewhere between this uniform draw and 1.0.

Some simulants are ending up with an RR **greater** than 1.0, which
seams weird. It looks like all such simulants are dead and in one of the
"waning effect" states, so my guess is that when you compute the
pipeline value for these simulants, i'ts just using the linear function
from the waning state, which will end up being > 1.0 because the dead
simulant has been "in that state" longer than the duration of the state.

In [57]:
pops[1].query("relative_risk < 1")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
16135,80.645822,dead,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2021-05-05 23:06:05.019770534,0.064682,0.527516,2048-11-28,0.245868,0.122597
21661,80.082013,dead,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2022-07-02 00:00:00.000000000,0.064075,0.527516,2048-11-28,0.243563,0.121448
21677,78.841921,dead,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2022-07-02 00:00:00.000000000,0.064075,0.527516,2048-11-28,0.243563,0.121448
21862,80.525519,dead,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2022-07-02 00:00:00.000000000,0.064075,0.527516,2048-11-28,0.243563,0.121448
22548,80.012122,dead,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2022-07-02 00:00:00.000000000,0.064075,0.527516,2048-11-28,0.243563,0.121448
...,...,...,...,...,...,...,...,...,...,...,...
123626,71.529870,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.031159,0.527516,2048-11-28,0.118440,0.059058
123654,75.197754,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.031159,0.527516,2048-11-28,0.118440,0.059058
123736,77.373435,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.031159,0.527516,2048-11-28,0.118440,0.059058
123760,76.023870,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.031159,0.527516,2048-11-28,0.118440,0.059058


In [58]:
pops[1].query("relative_risk > 1 ")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
1708,87.869944,dead,positive,waning_effect_long,alzheimers_mild_cognitive_impairment_state,2018-12-10 02:01:40.522973563,0.154397,1.236242,2048-11-28,0.250428,0.124871
12016,88.191754,dead,positive,waning_effect_long,alzheimers_mild_cognitive_impairment_state,2021-07-22 21:49:29.982230094,0.151324,1.236242,2048-11-28,0.245448,0.122388
13502,90.127423,dead,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2018-06-23 19:55:25.556567324,0.154919,1.236242,2048-11-28,0.251278,0.125295
19521,84.616974,dead,positive,waning_effect_long,alzheimers_disease_state,2017-08-23 21:46:22.742481443,0.155841,1.236242,2048-11-28,0.252774,0.126041
20057,77.510327,dead,positive,waning_effect_long,alzheimers_disease_state,2022-01-01 00:00:00.000000000,0.150780,1.236242,2048-11-28,0.244567,0.121948
...,...,...,...,...,...,...,...,...,...,...,...
96874,78.260796,dead,positive,waning_effect_short,alzheimers_mild_cognitive_impairment_state,2041-12-07 00:00:00.000000000,0.124884,1.377987,2048-11-28,0.181727,0.090614
100143,77.962159,dead,positive,waning_effect_short,alzheimers_disease_state,2042-06-07 00:00:00.000000000,0.122865,1.377987,2048-11-28,0.178788,0.089149
104385,71.126516,dead,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2043-06-06 00:00:00.000000000,0.118431,1.377987,2048-11-28,0.172336,0.085932
106801,81.927001,dead,positive,waning_effect_short,alzheimers_mild_cognitive_impairment_state,2043-12-05 00:00:00.000000000,0.108021,1.283490,2048-11-28,0.168760,0.084149


In [117]:
pops[1].query("relative_risk > 1 ").alive.unique()

array(['dead'], dtype=object)

# Check the ratio of the computed Weibull hazard to the MCI incidene rate in the sim

**Originally:** It looks like it's *usually* equal to 4.021402, whereas it should be
around 2.0. The engineers and Zeb confirmed that this is a bug where the
hazard function is getting multiplied by the step size of $\text{182
days} \approx \text{0.5 years}$ *twice* instead of once, so the
probability is about half as big as it should be.

**Update (2025-11-04):** This ratio is now close to 2.0 like it should be.

In [59]:
(pops[1]['mci_hazard'] / pops[1]['mci_inc_rate']).describe()

count    126279.000000
mean          2.041810
std           0.242672
min           0.510305
25%           2.005192
50%           2.005192
75%           2.005192
max           3.801198
dtype: float64

In [ ]:
# Hmm, the mean goes *up* when we filter to alive simulants
# Oh, maybe this makes sense because simulants with treatment will have
# smaller hazards than untreated simulants, making this ratio bigger
# than 2
temp = pops[1].query("alive=='alive'")
(temp['mci_hazard'] / temp['mci_inc_rate']).describe()

count    42507.000000
mean         2.090173
std          0.356913
min          2.005149
25%          2.005192
50%          2.005192
75%          2.005192
max          3.801198
dtype: float64

# Check the treatment effect size

The ratio between the treatment probabilities in the `full_effect`
states vs. the `susceptible` and `no_effect` treatment states should
match the hazard ratio.

In [61]:
# Filter to simulants in BBBM state
bbbm = pop1.query("alive=='alive' and ad_state=='alzheimers_blood_based_biomarker_state'")
bbbm

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
6624,73.924816,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2020-04-29 08:40:00.314983191,0.123120,1.0,2048-05-30
18161,80.849872,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2016-09-28 18:06:44.822523219,0.126416,1.0,2048-05-30
21568,84.568939,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2022-07-02 00:00:00.000000000,0.120958,1.0,2048-05-30
24140,91.985691,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-12-31 00:00:00.000000000,0.120443,1.0,2048-05-30
28836,82.781906,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-06-29 00:00:00.000000000,0.118847,1.0,2048-05-30
...,...,...,...,...,...,...,...,...,...
124174,92.211874,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124175,94.894029,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124176,93.454686,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124177,91.215896,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30


In [62]:
bbbm.treatment.unique()

array(['no_effect_never_treated', 'susceptible_to_treatment',
       'no_effect_after_long', 'waning_effect_long', 'full_effect_long',
       'no_effect_after_short', 'waning_effect_short',
       'full_effect_short', 'waiting_for_treatment'], dtype=object)

In [63]:
# Filter to simulants in BBBM state who are getting full treatment effect
bbbm_full_effect = bbbm.query("treatment in ['full_effect_long', 'full_effect_short']")
bbbm_full_effect

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
45113,64.706867,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2029-06-23,0.059553,0.527516,2048-05-30
52120,65.796393,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2030-12-21,0.058485,0.527516,2048-05-30
60528,65.190845,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2033-06-18,0.056535,0.527516,2048-05-30
60535,71.230452,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2033-06-18,0.056535,0.527516,2048-05-30
62598,65.173011,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2033-12-17,0.056115,0.527516,2048-05-30
...,...,...,...,...,...,...,...,...,...
121549,80.805957,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30
121676,79.992750,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30
121678,79.590437,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30
121694,80.236739,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30


In [64]:
# Filter to simulants in BBBM state with no treatment effect
bbbm_no_effect = bbbm.query(
    "treatment in ['susceptible_to_treatment', 'no_effect_never_treated', 'no_effect_after_long', 'no_effect_after_short']")
bbbm_no_effect

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
6624,73.924816,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2020-04-29 08:40:00.314983191,0.123120,1.0,2048-05-30
18161,80.849872,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2016-09-28 18:06:44.822523219,0.126416,1.0,2048-05-30
21568,84.568939,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2022-07-02 00:00:00.000000000,0.120958,1.0,2048-05-30
24140,91.985691,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-12-31 00:00:00.000000000,0.120443,1.0,2048-05-30
28836,82.781906,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-06-29 00:00:00.000000000,0.118847,1.0,2048-05-30
...,...,...,...,...,...,...,...,...,...
124174,92.211874,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124175,94.894029,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124176,93.454686,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30
124177,91.215896,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30


In [65]:
# Compute ratio of MCI incidence rates between the two groups
bbbm_full_effect.mci_inc_rate.mean() / bbbm_no_effect.mci_inc_rate.mean()

0.5605563116468726

In [66]:
# This should be a unique value that's close to the ratio above
# Yes, looks good!
bbbm_full_effect.relative_risk.unique()

array([0.52751591])

# Check hazard ratio in waning states

Hmm, the ratio of means for simulants in the `waning_effect_short` state
is close to the mean of the hazard ratios, but for simulants in the
`waning_effect_long` state, the ratio of means is larger than I would
expect -- closer to the maximum hazard ratio than to the mean. Could
this be due to not having any simulants who are near the end of the
`waning_effect_long` state?

I don't know if that makes sense, and I'm not sure what the right answer
should be...



In [67]:
bbbm_no_effect.mci_inc_rate.describe()

count    19011.000000
mean         0.075512
std          0.015023
min          0.050712
25%          0.064578
50%          0.075216
75%          0.085945
max          0.126416
Name: mci_inc_rate, dtype: float64

In [68]:
bbbm_waning_long = bbbm.query("treatment == 'waning_effect_long'")
bbbm_waning_long.relative_risk.describe()

count    528.000000
mean       0.680437
std        0.113228
min        0.553765
25%        0.580014
50%        0.658761
75%        0.737509
max        1.000000
Name: relative_risk, dtype: float64

In [ ]:
# Hmm, this is closer to the maximum hazard ratio than to the mean...
bbbm_waning_long.mci_inc_rate.mean() / bbbm_no_effect.mci_inc_rate.mean()

0.8872786096319314

In [70]:
# That's weird -- why do some simulants have an RR of 1.0 when they're
# still in the waning effect state?
bbbm_waning_short = bbbm.query("treatment == 'waning_effect_short'")
bbbm_waning_short.relative_risk.describe()

count    60.000000
mean      0.788957
std       0.128284
min       0.622013
25%       0.716510
50%       0.811006
75%       0.905503
max       1.000000
Name: relative_risk, dtype: float64

In [71]:
bbbm_waning_short.mci_inc_rate.mean() / bbbm_no_effect.mci_inc_rate.mean()

0.8197715737830461

# Look into RRs of 1.0 in the `waning_effect_short` state

It looks like this happens on the 4th time step of the state, out of 5
total steps. So maybe this is an off-by-one error?

In [72]:
bbbm_waning_short

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
95943,79.968503,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2041-06-08,0.082064,0.905503,2048-05-30
97045,82.502879,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2041-12-07,0.089162,1.000000,2048-05-30
101265,81.650264,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2042-12-06,0.085945,1.000000,2048-05-30
101392,81.559003,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2042-12-06,0.077824,0.905503,2048-05-30
102209,77.250712,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2042-12-06,0.085945,1.000000,2048-05-30
102295,79.164871,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2042-12-06,0.061580,0.716510,2048-05-30
102948,70.761961,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2043-06-06,0.068256,0.811006,2048-05-30
103136,78.539240,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2043-06-06,0.052350,0.622013,2048-05-30
103615,82.247898,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2043-06-06,0.068256,0.811006,2048-05-30
104517,76.498683,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2043-06-06,0.052350,0.622013,2048-05-30


In [73]:
# NOTE: Since I advanced the simulation further, the current time in the
# table below is in the future relative to the tables above. So some of
# the times will be in the future relative to the "current_time" in
# those tables.
sim.current_time

Timestamp('2050-05-28 00:00:00')

In [75]:
p = sim.get_population()
p.loc[bbbm_waning_short.index, [v for v in pop.columns if 'event_time' in v]]

,alzheimers_blood_based_biomarker_state_event_time,alzheimers_mild_cognitive_impairment_state_event_time,alzheimers_disease_state_event_time,no_effect_never_treated_event_time,waiting_for_treatment_event_time,susceptible_to_treatment_event_time,positive_test_event_time,full_effect_long_event_time,full_effect_short_event_time,waning_effect_long_event_time,waning_effect_short_event_time,no_effect_after_long_event_time,no_effect_after_short_event_time
95943,2041-06-08,NaT,NaT,NaT,2045-12-02,NaT,2045-12-02,NaT,2046-06-02,NaT,2046-12-01,NaT,2049-05-29
97045,2041-12-07,NaT,NaT,NaT,2045-06-03,NaT,2045-06-03,NaT,2045-12-02,NaT,2046-06-02,NaT,2048-11-28
101265,2042-12-06,2048-11-28,NaT,NaT,2045-06-03,NaT,2045-06-03,NaT,2045-12-02,NaT,2046-06-02,NaT,2048-11-28
101392,2042-12-06,NaT,NaT,NaT,2045-12-02,NaT,2045-12-02,NaT,2046-06-02,NaT,2046-12-01,NaT,2049-05-29
102209,2042-12-06,NaT,NaT,NaT,2045-06-03,NaT,2045-06-03,NaT,2045-12-02,NaT,2046-06-02,NaT,2048-11-28
102295,2042-12-06,NaT,NaT,NaT,2046-12-01,NaT,2046-12-01,NaT,2047-06-01,NaT,2047-11-30,NaT,2050-05-28
102948,2043-06-06,NaT,NaT,NaT,2046-06-02,NaT,2046-06-02,NaT,2046-12-01,NaT,2047-06-01,NaT,2049-11-27
103136,2043-06-06,NaT,NaT,NaT,2047-06-01,NaT,2047-06-01,NaT,2047-11-30,NaT,2048-05-30,NaT,NaT
103615,2043-06-06,NaT,NaT,NaT,2046-06-02,NaT,2046-06-02,NaT,2046-12-01,NaT,2047-06-01,NaT,2049-11-27
104517,2043-06-06,NaT,NaT,NaT,2047-06-01,NaT,2047-06-01,NaT,2047-11-30,NaT,2048-05-30,NaT,NaT


In [78]:
# The simulants with an RR of 1.0 are on the 4th timestep of the waning state
(pd.Timestamp('2048-05-30') - pd.Timestamp('2046-06-02')).days / 182

4.0

In [77]:
p.loc[bbbm_waning_long.index, [v for v in pop.columns if 'event_time' in v]]

,alzheimers_blood_based_biomarker_state_event_time,alzheimers_mild_cognitive_impairment_state_event_time,alzheimers_disease_state_event_time,no_effect_never_treated_event_time,waiting_for_treatment_event_time,susceptible_to_treatment_event_time,positive_test_event_time,full_effect_long_event_time,full_effect_short_event_time,waning_effect_long_event_time,waning_effect_short_event_time,no_effect_after_long_event_time,no_effect_after_short_event_time
32201,2025-06-28,NaT,NaT,NaT,2041-06-08,NaT,2041-06-08,2041-12-07,NaT,2046-12-01,NaT,NaT,NaT
32728,2025-12-27,NaT,NaT,NaT,2038-06-12,NaT,2038-06-12,2038-12-11,NaT,2043-12-05,NaT,NaT,NaT
33921,2025-12-27,NaT,NaT,NaT,2035-06-16,NaT,2035-06-16,2035-12-15,NaT,2040-12-08,NaT,2049-11-27,NaT
39675,2027-12-25,NaT,NaT,NaT,2036-06-14,NaT,2036-06-14,2036-12-13,NaT,2041-12-07,NaT,NaT,NaT
43254,2028-12-23,NaT,NaT,NaT,2040-06-09,NaT,2040-06-09,2040-12-08,NaT,2045-12-02,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99496,2042-06-07,NaT,NaT,NaT,2042-12-06,NaT,NaT,2043-06-06,NaT,2048-05-30,NaT,NaT,NaT
99504,2042-06-07,NaT,NaT,NaT,2042-12-06,NaT,NaT,2043-06-06,NaT,2048-05-30,NaT,NaT,NaT
100256,2042-06-07,NaT,NaT,NaT,2042-12-06,NaT,NaT,2043-06-06,NaT,2048-05-30,NaT,NaT,NaT
100294,2042-06-07,NaT,NaT,NaT,2042-12-06,NaT,NaT,2043-06-06,NaT,2048-05-30,NaT,NaT,NaT


In [ ]:
# Now we have some simulants who have reached the end of the waning_effect_long state
p.query("no_effect_after_long_event_time.notna()")

,tracked,alive,age,exit_time,entrance_time,location,sex,years_of_life_lost,cause_of_death,alzheimers_disease_and_other_dementias,...,full_effect_short_event_count,full_effect_short_event_time,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_time,waning_effect_short_event_count,no_effect_after_long_event_time,no_effect_after_long_event_count,no_effect_after_short_event_count,no_effect_after_short_event_time
7337,True,alive,86.204683,NaT,2021-07-03,United States of America,Female,0.000000,not_dead,alzheimers_disease_state,...,0,NaT,2035-12-15,1,NaT,0,2044-12-03,1,0,NaT
18415,True,dead,79.595544,2049-11-27,2021-07-03,United States of America,Female,14.418401,alzheimers_disease_state,alzheimers_disease_state,...,0,NaT,2036-12-13,1,NaT,0,2045-12-02,1,0,NaT
20045,True,dead,81.941549,2045-12-02,2022-01-01,NaN,Female,12.808469,alzheimers_disease_state,alzheimers_disease_state,...,0,NaT,2035-12-15,1,NaT,0,2044-12-03,1,0,NaT
20120,True,dead,92.896251,2048-05-30,2022-01-01,NaN,Female,8.574440,other_causes,alzheimers_mild_cognitive_impairment_state,...,0,NaT,2036-12-13,1,NaT,0,2045-12-02,1,0,NaT
20129,True,dead,94.153302,2048-05-30,2022-01-01,NaN,Female,8.524555,other_causes,alzheimers_mild_cognitive_impairment_state,...,0,NaT,2035-12-15,1,NaT,0,2044-12-03,1,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68147,True,alive,87.577956,NaT,2034-12-16,NaN,Male,0.000000,not_dead,alzheimers_disease_state,...,0,NaT,2040-12-08,1,NaT,0,2049-11-27,1,0,NaT
68964,True,alive,87.019112,NaT,2035-06-16,NaN,Female,0.000000,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,2041-06-08,1,NaT,0,2050-05-28,1,0,NaT
69105,True,alive,92.404068,NaT,2035-06-16,NaN,Female,0.000000,not_dead,alzheimers_disease_state,...,0,NaT,2041-06-08,1,NaT,0,2050-05-28,1,0,NaT
69167,True,alive,91.362562,NaT,2035-06-16,NaN,Female,0.000000,not_dead,alzheimers_blood_based_biomarker_state,...,0,NaT,2041-06-08,1,NaT,0,2050-05-28,1,0,NaT


# Compute range of expected RRs in waning effect states

In [101]:
# Range of expected RR values in 'waning_effect_short' state
RR = bbbm_full_effect.relative_risk.unique()[0] # R_d in the docs
s = 5 # time steps in waning short
[RR + i * (1 - RR) / s for i in range(s+2)]

[0.5275159063282223,
 0.6220127250625779,
 0.7165095437969333,
 0.8110063625312889,
 0.9055031812656444,
 1.0,
 1.0944968187343556]

In [99]:
bbbm_waning_short.relative_risk.unique()

array([0.90550318, 1.        , 0.71650954, 0.81100636, 0.62201273])

In [81]:
bbbm_waning_short.relative_risk.describe()

count    60.000000
mean      0.788957
std       0.128284
min       0.622013
25%       0.716510
50%       0.811006
75%       0.905503
max       1.000000
Name: relative_risk, dtype: float64

In [82]:
# Range of expected RR values in 'waning_effect_long' state
s = 9 * 2
[RR + i * (1 - RR) / s for i in range(s+2)]

[0.5275159063282223,
 0.5537650226433211,
 0.5800141389584198,
 0.6062632552735185,
 0.6325123715886174,
 0.6587614879037161,
 0.6850106042188149,
 0.7112597205339136,
 0.7375088368490124,
 0.7637579531641112,
 0.7900070694792098,
 0.8162561857943087,
 0.8425053021094074,
 0.8687544184245062,
 0.895003534739605,
 0.9212526510547037,
 0.9475017673698025,
 0.9737508836849013,
 1.0,
 1.0262491163150989]

In [104]:
sorted(bbbm_waning_long.relative_risk.unique())

[0.5537650226433211,
 0.5800141389584198,
 0.6062632552735185,
 0.6325123715886174,
 0.6587614879037161,
 0.6850106042188149,
 0.7112597205339136,
 0.7375088368490124,
 0.7637579531641112,
 0.79000706947921,
 0.8162561857943087,
 0.8425053021094074,
 0.8687544184245062,
 0.895003534739605,
 0.9212526510547037,
 0.9475017673698025,
 0.9737508836849011,
 1.0]

In [83]:
bbbm_waning_long.relative_risk.describe()

count    528.000000
mean       0.680437
std        0.113228
min        0.553765
25%        0.580014
50%        0.658761
75%        0.737509
max        1.000000
Name: relative_risk, dtype: float64

# Look into simulants who transition into a waning state

Simulants' hazard ratio increases on the same time step they enter the
waning state, which is contrary to the docs, which say it should only
start increasing on the next time step.

In [111]:
pops[1].treatment.unique()

array(['susceptible_to_treatment', 'no_effect_never_treated',
       'waiting_for_treatment', 'waning_effect_long',
       'no_effect_after_long', 'full_effect_long',
       'no_effect_after_short', 'waning_effect_short',
       'full_effect_short'], dtype=object)

In [112]:
full_to_waning_short = (pops[0].treatment == 'full_effect_short') & (pops[1].treatment == 'waning_effect_short')
pops[0].loc[full_to_waning_short]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
96531,64.245742,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2041-12-07,0.047035,0.527516,2048-05-30,0.178788,0.089149
109483,72.173377,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2044-12-03,0.041046,0.527516,2048-05-30,0.156024,0.077798
111079,80.419511,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2044-12-03,0.041046,0.527516,2048-05-30,0.156024,0.077798
111636,69.701912,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2045-06-03,0.039677,0.527516,2048-05-30,0.150822,0.075204
112853,66.985324,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2045-06-03,0.039677,0.527516,2048-05-30,0.150822,0.075204
113195,80.064593,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2045-06-03,0.039677,0.527516,2048-05-30,0.150822,0.075204
113654,68.024749,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2045-12-02,0.038117,0.527516,2048-05-30,0.144892,0.072247
115700,65.644602,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2046-06-02,0.036291,0.527516,2048-05-30,0.137951,0.068786
115825,69.409023,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2046-06-02,0.036291,0.527516,2048-05-30,0.137951,0.068786
117390,80.106296,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2046-06-02,0.036291,0.527516,2048-05-30,0.137951,0.068786


In [113]:
pops[1].loc[full_to_waning_short]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
96531,64.744031,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2041-12-07,0.056372,0.622013,2048-11-28,0.181727,0.090614
109483,72.671666,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2044-12-03,0.049842,0.622013,2048-11-28,0.160676,0.080118
111079,80.917800,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2044-12-03,0.049842,0.622013,2048-11-28,0.160676,0.080118
111636,70.200200,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2045-06-03,0.048399,0.622013,2048-11-28,0.156024,0.077798
112853,67.483613,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2045-06-03,0.048399,0.622013,2048-11-28,0.156024,0.077798
113195,80.562881,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2045-06-03,0.048399,0.622013,2048-11-28,0.156024,0.077798
113654,68.523038,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2045-12-02,0.046785,0.622013,2048-11-28,0.150822,0.075204
115700,66.142891,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2046-06-02,0.044946,0.622013,2048-11-28,0.144892,0.072247
115825,69.907312,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2046-06-02,0.044946,0.622013,2048-11-28,0.144892,0.072247
117390,80.604585,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2046-06-02,0.044946,0.622013,2048-11-28,0.144892,0.072247


# Check treatment state durations

Looks good based on recorded event times, but it looks like I haven't
run the sim long enough yet to check the `waning_effect_long` state.

In [84]:
p = sim.get_population()

In [85]:
completed_short_treatment = p.query("no_effect_after_short_event_time.notna()")
(
    completed_short_treatment['no_effect_after_short_event_time']
    - completed_short_treatment['waning_effect_short_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([2.5])

In [86]:
entered_waning_short_treatment = p.query("waning_effect_short_event_time.notna()")
(
    entered_waning_short_treatment['waning_effect_short_event_time']
    - entered_waning_short_treatment['full_effect_short_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([0.5])

In [87]:
# I haven't run the sim long enough to observe anyone complete long
# treatment yet
completed_long_treatment = p.query("no_effect_after_long_event_time.notna()")
(
    completed_long_treatment['no_effect_after_long_event_time']
    - completed_long_treatment['waning_effect_long_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([9.])

In [88]:
entered_waning_long_treatment = p.query("waning_effect_long_event_time.notna()")
(
    entered_waning_long_treatment['waning_effect_long_event_time']
    - entered_waning_long_treatment['full_effect_long_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([5.])

# Compare calculated Weibull hazard to hazard rate value from pipeline

In [ ]:
# Among those with no treatment effect, the ratio should be the same for
# everyone.
# Hmm, now there's less variation than before. This seems more likely to
# just be rounding errors
ratio = (mci_incidence_hazard(bbbm_no_effect) / bbbm_no_effect['mci_inc_rate']).rename('haz_to_prob_ratio')
ratio.describe()

count    1.901100e+04
mean     2.005192e+00
std      2.105130e-07
min      2.005165e+00
25%      2.005192e+00
50%      2.005192e+00
75%      2.005192e+00
max      2.005192e+00
Name: haz_to_prob_ratio, dtype: float64

In [90]:
# Look into values that are smaller than expected
bbbm_no_effect.join(mci_incidence_hazard(bbbm_no_effect)).join(ratio).loc[ratio < 2.00519]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,haz_to_prob_ratio
6624,73.924816,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2020-04-29 08:40:00.314983191,0.123120,1.0,2048-05-30,0.246876,2.005165
18161,80.849872,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2016-09-28 18:06:44.822523219,0.126416,1.0,2048-05-30,0.253487,2.005183


In [92]:
# The ratio should be larger than the value above for people getting treatment
ratio = (mci_incidence_hazard(bbbm) /  bbbm['mci_inc_rate']).rename('haz_to_prob_ratio')
bbbm.join(mci_incidence_hazard(bbbm)).join(ratio).loc[ratio > 2.0052]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,haz_to_prob_ratio
32201,84.547606,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2025-06-28,0.074472,0.632512,2048-05-30,0.236090,3.170203
32728,79.670622,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2025-12-27,0.092566,0.790007,2048-05-30,0.234951,2.538196
33921,92.623737,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2025-12-27,0.111020,0.947502,2048-05-30,0.234951,2.116294
39675,72.604745,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2027-12-25,0.102743,0.895004,2048-05-30,0.230188,2.240430
43254,68.490792,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2028-12-23,0.077776,0.685011,2048-05-30,0.227670,2.927243
...,...,...,...,...,...,...,...,...,...,...,...
121549,80.805957,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30,0.118440,3.801198
121676,79.992750,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30,0.118440,3.801198
121678,79.590437,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30,0.118440,3.801198
121694,80.236739,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2047-06-01,0.031159,0.527516,2048-05-30,0.118440,3.801198


# Check whether new simulants are always in BBBM state

Yes, looks good. Too bad I didn't check this in previous versions of the
model since it was apparently broken...

In [105]:
pops[0]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.123728,1.0,2048-05-30,0.248094,0.123707
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.122090,1.0,2048-05-30,0.244813,0.122071
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.126285,1.0,2048-05-30,0.253226,0.126266
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.126686,1.0,2048-05-30,0.254030,0.126667
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-01-03 12:31:22.341663442,0.127057,1.0,2048-05-30,0.254772,0.127037
...,...,...,...,...,...,...,...,...,...,...,...
124174,92.211874,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124175,94.894029,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124176,93.454686,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705
124177,91.215896,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-11-30 00:00:00.000000000,0.050712,1.0,2048-05-30,0.101688,0.050705


In [108]:
new_sims = pops[1].index.difference(pops[0].index)
pops[1].loc[new_sims]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,mci_prob
124179,44.034397,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
124180,43.075477,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
124181,43.430643,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
124182,40.646644,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
124183,40.781791,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
...,...,...,...,...,...,...,...,...,...,...,...
126274,93.013379,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
126275,91.094237,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
126276,92.981823,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705
126277,91.072722,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2048-05-30,0.050712,1.0,2048-11-28,0.101688,0.050705


In [109]:
pops[1].loc[new_sims].ad_state.unique()

array(['alzheimers_blood_based_biomarker_state'], dtype=object)

In [110]:
for i in range(1, len(pops)):
    new_sims = pops[i].index.difference(pops[i-1].index)
    print(pops[i].loc[new_sims, 'ad_state'].unique())

['alzheimers_blood_based_biomarker_state']
['alzheimers_blood_based_biomarker_state']
['alzheimers_blood_based_biomarker_state']
['alzheimers_blood_based_biomarker_state']


In [114]:
sim.current_time

Timestamp('2050-05-28 00:00:00')